# Проект "Определение стоимости автомобилей"

### Цель проекта:

В нашем распоряжении исторические данные по автомобилям:
+ технические характеристики, 
+ комплектации и цены автомобилей. 

Нам нужно построить модель для определения стоимости.  

Заказчику важны:  
+ качество предсказания;
+ скорость предсказания;
+ время обучения.

### Описание даных

Данные находятся в файле autos.csv.   

Признаки: 

+ DateCrawled — дата скачивания анкеты из базы
+ VehicleType — тип автомобильного кузова
+ RegistrationYear — год регистрации автомобиля
+ Gearbox — тип коробки передач
+ Power — мощность (л. с.)
+ Model — модель автомобиля
+ Kilometer — пробег (км)
+ RegistrationMonth — месяц регистрации автомобиля
+ FuelType — тип топлива
+ Brand — марка автомобиля
+ NotRepaired — была машина в ремонте или нет
+ DateCreated — дата создания анкеты
+ NumberOfPictures — количество фотографий автомобиля
+ PostalCode — почтовый индекс владельца анкеты (пользователя)
+ LastSeen — дата последней активности пользователя  

Целевой признак:  
+ Price — цена (евро)

# Содержание проекта

[1. Загрузка и подготовка данных](#1)  
[2. Обучение моделей](#2)  
[3. Анализ качества и скорости работы моделей](#3)  

### Установка необходимых пакетов

In [ ]:
pip install optuna

### Импорт необходимых библиотек

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

import lightgbm
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb

from xgboost import XGBRegressor

import warnings
import random

# 1. Загрузка и подготовка данных <a id="1"></a>

In [ ]:
display(autos.info())
display(autos.describe())
display(autos.head())

Удалим столбцы, не несущие смысловой нагрузки при определении стоимости автомобиля:

In [ ]:
autos.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1, inplace=True)

Удалим строки с нулевыми значениями цены, т.к. в рамках данного проекта эти цены аномальны, могут повлиять на качество итогового прогнозирования. Предположительно, данные строки либо ошибочны, либо отражают какой-то рыночный механизм того региона, в котором собиралась база данных.  Т.к. выяснить у заказчика природу этих данных мы не можем, поэтому просто удалим. Аномально большие зачения стоимости оставлены без изменений, т.к. не знаем реалий исследуемого рынка и "ценности" конкретного авто.

In [ ]:
autos = autos.query('Price > 0')

Удалим строки с явно некорректными датами регистрации (выпуска автомобиля)

In [ ]:
autos.query('RegistrationYear < 1900 or RegistrationYear > 2021')['RegistrationYear'].count()

In [ ]:
autos.RegistrationYear.unique()

In [ ]:
autos = autos.query('RegistrationYear > 1900 and RegistrationYear <= 2021')

Оценим количество некорректных значений месяца регистрации 

In [ ]:
autos.RegistrationMonth.unique()

In [ ]:
autos.query('RegistrationMonth == 0')['RegistrationMonth'].count()

In [ ]:
autos.RegistrationMonth.value_counts()

Т.к. кол-во значительно, удаление нецелесообразно, заполним рандомно

In [ ]:
def rand_month(row):
    """
    Функция рандомайзера для значений (1, 12):
    row - строка DF 
    Возвращает значение от 1 до 12
    """
    return random.randint(1, 12)

In [ ]:
warnings.filterwarnings("ignore")
month_zero = autos.query('RegistrationMonth == 0')
month_zero['RegistrationMonth'] = month_zero.apply(rand_month, axis=1)


In [ ]:
autos.loc[month_zero.index, 'RegistrationMonth'] = month_zero.loc[:, 'RegistrationMonth']

Обработаем некорректные значения мощности, заполним значения меньше 10 л.с. и больше 500 л.с. медианным значением.

In [ ]:
median = autos.query('Power > 10 and Power < 500')['Power'].median()
print(median)

In [ ]:
autos.loc[autos.loc[:, 'Power'] < 10, 'Power'] = median
autos.loc[autos.loc[:, 'Power'] > 500, 'Power'] = median

Удалим дубликаты

In [ ]:
autos.duplicated().sum()

In [ ]:
autos = autos.drop_duplicates().reset_index(drop=True)

## Вывод

На данном этапе обработали очевидно аномальные значения и дубликаты. Первые обусловлены скорее всего ошибками пользователей, вторые дублирование продажи одного и того же автомобиля самим продавцом или агентами.  
Кроме того, произведен предварительный анализ предоставленых данных, удалены те, которые не представляют интереса в будущем моделировании.

### Обработка пропусков:

### VehicleType

Заполним пустые значения типа кузова. Создадим "словарь" Брэнд-Модель-Тип кузова-Количество автомобилей, с сортировкой по убыванию, по количеству автомобилей. В качестве искомого типа кузова примем самое частое значение типа кузова для автомобиля аналогичного бренда и модели.

In [ ]:
model_type_pivot = autos.pivot_table(
    index=['Brand', 'Model', 'VehicleType'], values='Price', aggfunc='count')

Сортируем сводную таблицу по убыванию количества автомобилей заданного бренда, модели и типа кузова:

In [ ]:
model_type_pivot = model_type_pivot.groupby(
    level=['Brand', 'Model'], group_keys=False
).apply(pd.DataFrame.sort_values, by='Price', ascending=False)

model_type_pivot.columns = ['Count']

display(model_type_pivot)

Оставляем в таблице только самые частые значения `VehicleType` для каждого бренда и модели (первую строку каждой группы):

In [ ]:
df_temp = model_type_pivot.reset_index().drop(
    'Count', axis=1).groupby(['Brand', 'Model']).first()
display(df_temp)

In [ ]:
def get_model2(string, dict = df_temp.reset_index()):
    """
    Функция для определения типа кузова автомобиля по Брэнду и Модели:
    string - строка DF, в которой необходимо заполнить тип кузова
    dict - словарь брендов, моделей, кузовов а/м
    Возвращает тип кузова.
    """
    for index, row in dict.iterrows():
        if (row['Brand'] == string['Brand']) & (row['Model'] == string['Model']):
            return row['VehicleType']


Создадим отдельный датафрейм с записями, подлежащими корректировке.

In [ ]:
type_nan = autos.query('VehicleType != VehicleType and Brand == Brand and Model == Model') 
display(type_nan)

In [ ]:
%%time
# Заполняем отсутствующие значения
warnings.filterwarnings("ignore")
type_nan['VehicleType']  = type_nan.apply(get_model2, axis = 1)

Wall time: 16min 46s

In [ ]:
autos.loc[type_nan.index, 'VehicleType'] = type_nan.loc[:, 'VehicleType']

In [ ]:
display(autos.head())

Оставшиеся пустые значения типа кузова заменим на `others`

In [ ]:
autos.VehicleType = autos.VehicleType.fillna('other')

Прочие пропущенные параметры автомобиля заменим либо на наиболее употребимое, либо на `others`, если это уместно и нет однозначного параметра, по которому можно точно установить пропущенное значение. 

In [ ]:
autos.Gearbox.value_counts()

In [ ]:
autos.Gearbox = autos.Gearbox.fillna('manual')

In [ ]:
autos.Model.value_counts()

In [ ]:
autos.Model = autos.Model.fillna('other')

In [ ]:
autos.FuelType.value_counts()

In [ ]:
autos.FuelType = autos.FuelType.fillna('other')

In [ ]:
autos.NotRepaired.value_counts()

In [ ]:
autos.NotRepaired = autos.NotRepaired.fillna('no')

In [ ]:
autos.reset_index(drop=True)
autos.info()

Обработка пропусков завершена

### Кодирование категориальных признаков

Столбец `Model` закодируем через `LabelEncoder`, остальные через OHE.

In [ ]:
encoder = LabelEncoder()
encoder.fit(autos['Model'])
autos['Model'] = encoder.transform(autos['Model'])

autos = pd.get_dummies(autos, drop_first=True)

In [ ]:
display(autos.head())

In [ ]:
autos.reset_index(drop=True)
autos.info()

### Масштабирование признаков

In [ ]:
features = autos.drop('Price', axis=1)
target = autos['Price']

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, target,  test_size = 0.25, random_state = 12345)

In [ ]:
warnings.filterwarnings("ignore")
numeric = ['RegistrationYear', 'Power', 'Kilometer','Model', 'RegistrationMonth', 'NumberOfPictures', 'PostalCode']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

display(features_train.head())

## Вывод

Проведена предобработка данных, обработаны пропущенные значения, произведено кодирование категориальных признаков и выполнено масштабирование.

# 2. Обучение моделей <a id="2"></a>

Обучим несколько моделей, предварительно оценим результаты, по итогам подбора параметров.   


In [ ]:
def rmse(target, predict):
    """
    Функция расчета RMSE:
    target - целевой признак
    predict - предсказание
    Возвращает RMSE.
    """
    return (mean_squared_error(target, predict)) ** 0.5

In [ ]:
# Создадим метрику 
custom_scorer = make_scorer(rmse, greater_is_better=False)

### LinearRegression

In [ ]:
%%time
model = LinearRegression()
RMSE = pd.Series(cross_val_score(
    model, features_train, target_train, scoring=custom_scorer, cv=3)).max()
print('RMSE', RMSE)

### RandomForest Regressor

In [ ]:
%%time
model = RandomForestRegressor(random_state=12345, criterion='mse')
# уже оптимизированные параметры, для экономии времени расчета
parameters = {'n_estimators': [40], 'max_depth': [50]}
grid = GridSearchCV(model, parameters, scoring=custom_scorer, cv=3)
grid.fit(features_train, target_train)

print('Оптимальные параметры модели:', grid.best_params_)
print('RMSE', grid.best_score_)

Результат предыдущего запуска:  
Оптимальные параметры модели: {'max_depth': 50, 'n_estimators': 40}  
RMSE -1734.3282855473062  
CPU times: user 7min 50s, sys: 2.06 s, total: 7min 52s  
Wall time: 7min 53s


### XGBoost

In [ ]:
%%time
model = XGBRegressor(random_state=12345)
# уже оптимизированные параметры, для экономии времени расчета
parameters = {'n_estimators': [25], 'max_depth': [12]}
grid = GridSearchCV(model, parameters, scoring=custom_scorer, cv=3)
grid.fit(features_train, target_train)

print('Оптимальные параметры модели:', grid.best_params_)
rmse = grid.best_score_
print('RMSE', rmse)

Результат предыдущего запуска:  
Оптимальные параметры модели: {'max_depth': 12, 'n_estimators': 25}  
RMSE -1715.7998195591829  
CPU times: user 6min 14s, sys: 5.34 s, total: 6min 20s  
Wall time: 1min 41s

Подбор параметров с помощью `GridSearchCV` для моделей `RandomForestRegressor` и `XGBRegressor` с большим диапазоном параметров занимает очень много времени, поэтому оставлены оптимизированные параметры и результаты последнего запуска. 

### LightGBM

Для подбора параметров `LightGBM` используем фреймворк подбора гиперпараметров `Optuna`, рекомендованный документацией LightGBM  и тюнер `LightGBMTunerCV`

In [ ]:
%%time
dtrain = lgb.Dataset(features_train, label=target_train)
params = {
    "objective": "regression",
    "metric": "rmse",
    "verbosity": -1,
    "boosting_type": "gbdt"
    
}

tuner = lgb.LightGBMTunerCV(params, dtrain, verbose_eval=250, early_stopping_rounds=250, folds=KFold(n_splits=3))

tuner.run()

best_score = tuner.best_score
print("RMSE:", best_score)
best_params = tuner.best_params
print("Оптимальные параметры модели:", best_params)

Результат предыдущего запуска:  
RMSE: 1635.2636092534503  
Оптимальные параметры модели: {'objective': 'regression', 'metric': 'rmse', 'verbosity': -1, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 143, 'feature_fraction': 0.7, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 5}

## Вывод

По итогам подбора параметров, `RandomForestRegressor`, `XGBRegressor`, `LightGBM` показывают схожие результаты по метрике RMSE, в диапазоне 1640-1740.  
Далее на полученных оптимизированных параметрах обучим эти модели и сделаем прогноз на тестовой выборке.  
Также на этом этапе оценим скорость работы моделей.

# 3. Анализ качества и скорости работы моделей <a id="1"></a>

In [ ]:
def rmse_test(model, features_train=features_train, target_train=target_train, features_test=features_test, target_test=target_test):
    """
    Функция расчета RMSE для тестовой выборки:
    model - модель
    target - целевой признак
    features - признаки
    Возвращает RMSE.
    """
    model.fit(features_train, target_train)
    predicted_test = model.predict(features_test)
    mse = mean_squared_error(target_test, predicted_test)
    return mse ** 0.5

In [ ]:
%%time
model = RandomForestRegressor(random_state=12345, max_depth=50, n_estimators=40)
print("RMSE =", rmse_test(model))

In [ ]:
%%time
model = XGBRegressor(random_state=12345, max_depth=12, n_estimators=25)
print("RMSE =", rmse_test(model))

In [ ]:
%%time
model = LGBMRegressor(random_state=12345, num_leaves=202, min_child_samples=5)
print("RMSE =", rmse_test(model))

## Вывод

Модель `LGBMRegressor` на тестовых выборках также показала наилучшие результаты как по значению искомой метрики (RMSE=1669), так и по скорости обучения (6,2 сек).

# Общий вывод

В процессе выполнения проекта были оценены различные модели с позиции скорости их обучения и точности предсказаний стоимости автомобиля.  
Результаты показывают, что модели на основе градиентного бустинга обучаются значительно быстрее моделей линейной регрессии и случайного леса на предоставленных данных.  
Итоговую модель LightGBM, показывающую наилучшие результаты применим для целей бизнеса - предсказания стоимости автомобиля сервиса "Не бит, не крашен".  